In [64]:
import csv
import numpy as np
import os
import pickle

In [43]:
companies_to_etfs = {}
with open("pickles/companies_to_etfs.pkl", 'rb') as file:
    companies_to_etfs = pickle.load(file)

In [155]:
company = "A"
companies_to_etfs[company] = {}

folder_path = 'etf_csvs'
for filename in os.listdir(folder_path):
    if filename == ".DS_Store":
        continue
    with open(os.path.join(folder_path, filename), newline='') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',')
        next(csvreader)
        for row in csvreader:
            etf = row[0]
            weight = float(row[3][:-1])
            if etf not in companies_to_etfs[company]:
                companies_to_etfs[company][etf] = weight
            else:
                companies_to_etfs[company][etf] += weight

print(companies_to_etfs[company])

{'HLAL': 0.18, 'CDEI': 0.18, 'LRGF': 0.18, 'QUAL': 0.18, 'OUSA': 0.17, 'JCTR': 0.17, 'EUSA': 0.17, 'SIZE': 0.17, 'USSG': 0.17, 'SUSL': 0.17, 'PINK': 0.17, 'DSI': 0.16, 'FAD': 0.16, 'DFSU': 0.16, 'ILCG': 0.16, 'SPUS': 0.16, 'QRFT': 0.15, 'NACP': 0.15, 'HKND': 0.14, 'VONG': 0.14, 'JHML': 0.14, 'YPS': 0.14, 'GGUS': 0.13, 'XYLE': 0.13, 'LCTU': 0.13, 'SPXT': 0.13, 'WOMN': 0.13, 'GSLC': 0.13, 'SSPY': 0.13, 'EFIV': 0.12, 'IWF': 0.12, 'SNPE': 0.12, 'USCA': 0.12, 'MGC': 0.12, 'USCL': 0.12, 'QUS': 0.12, 'KOMP': 0.12, 'STXG': 0.12, 'ESGU': 0.11, 'JQUA': 0.11, 'NZUS': 0.11, 'SPXN': 0.11, 'DUBS': 0.11, 'CVLC': 0.1, 'RAFE': 0.1, 'VOO': 0.1, 'JUST': 0.1, 'XYLD': 0.1, 'XVV': 0.1, 'VV': 0.1, 'XYLG': 0.1, 'DFAC': 0.1, 'DCOR': 0.1, 'DFAU': 0.1, 'QVML': 0.1, 'XRMI': 0.1, 'PBP': 0.1, 'ESGV': 0.1, 'IWS': 0.1, 'SPXE': 0.1, 'SPYX': 0.09, 'NSPI': 0.09, 'PTLC': 0.09, 'XTR': 0.09, 'VONE': 0.09, 'BKLC': 0.09, 'IETC': 0.09, 'GSUS': 0.09, 'SPY': 0.09, 'SPYI': 0.09, 'GPIX': 0.09, 'IVV': 0.09, 'SPLG': 0.09, 'ONOF': 0

In [156]:
with open("pickles/companies_to_etfs.pkl", 'wb') as file:
     pickle.dump(companies_to_etfs, file)

In [157]:
etfs_dict = {}

for company in companies_to_etfs:
    for etf in companies_to_etfs[company]:
        if etf not in etfs_dict:
            etfs_dict[etf] = [0,0]
        etfs_dict[etf][0] += 1
        etfs_dict[etf][1] += companies_to_etfs[company][etf]

In [ ]:
covered_etfs = set({"XLK", "XLC", "PPH", "IYC", "XLV"})
covered_companies = set()
new_etfs_dict = {}

for company in companies_to_etfs:
    for etf in covered_etfs:
        if etf in companies_to_etfs[company]:
            covered_companies.add(company)

# covered_companies = set(["PYPL", "MA", "AXP"])
            
for company in companies_to_etfs:
    for etf in companies_to_etfs[company]:
        if etf not in new_etfs_dict:
            new_etfs_dict[etf] = [0,0,0]
        new_etfs_dict[etf][0] += 1
        if company not in covered_companies:
            new_etfs_dict[etf][1] += companies_to_etfs[company][etf]
        else:
            new_etfs_dict[etf][2] += companies_to_etfs[company][etf]
        # new_etfs_dict[etf][0] += 1
        # if company not in covered_companies:
        #     new_etfs_dict[etf][2] += companies_to_etfs[company][etf]
        # else:
        #     new_etfs_dict[etf][1] += companies_to_etfs[company][etf]

etfs = list(new_etfs_dict)
weight = [new_etfs_dict[etf][1] for etf in etfs]

argsorted = np.argsort(weight)[::-1]

for i in argsorted:
    print(etfs[i], new_etfs_dict[etfs[i]])

In [ ]:
# "XLK" 26%, "XLC" 6%, "PPH" 12%, "IYC" 10%, "XLV" 22%, "IYJ" 18%, "QTUM" 6%

In [364]:
# len(covered_companies)

for company in companies_to_etfs:
    if "XLK" in companies_to_etfs[company] and "QTUM" in companies_to_etfs[company]:
        print(company)

IBM
MSFT
INTC
ACN
QCOM
NVDA
TXN
NXPI
AMD


In [189]:
from ortools.linear_solver import pywraplp
import time

solver = pywraplp.Solver.CreateSolver('SCIP')

variables = {}

for etf in etfs_dict:
    if etfs_dict[etf][0] == 1:
        continue
    variables[etf] = solver.IntVar(0.0, 1.0, etf)

for company in companies_to_etfs:
    new_constraint = 0
    for etf in companies_to_etfs[company]:
        if etf in variables:
            new_constraint += variables[etf]
    solver.Add(new_constraint >= 1)

obj_func = 0
for etf in variables:
    obj_func += variables[etf]*etfs_dict[etf][0]*(100 - etfs_dict[etf][1])

solver.Minimize(obj_func)

In [190]:
print(solver.NumVariables())
print(solver.NumConstraints())

924
50


In [191]:
start = time.time()
status = solver.Solve()
end = time.time()
print('elapsed time: ' + str(end - start))

elapsed time: 0.02744293212890625


In [196]:
if status == pywraplp.Solver.OPTIMAL:
    for etf in variables:
        if variables[etf].solution_value() > 0:
            print(etf, variables[etf].solution_value())
else:
    print('The problem does not have an optimal solution.')

MMTM 1.0
AVSD 1.0
AVEM 1.0


In [195]:
etfs_dict["AVEM"]

[2, 0.15000000000000002]